In [2]:
import pandas as pd

answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [3]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])

In [4]:
a2=train[train['target']==1]

In [5]:
a2=a2[['qid','uid','time']]
a2.columns=['qid','uid','atime']

In [6]:
answer_info=answer_info[['aid','qid','uid','atime']]

In [7]:
answer_info=pd.concat([answer_info,a2])

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
answer_info.drop(['aid'],axis=1,inplace=True)

In [9]:
answer_info=answer_info.drop_duplicates(subset=['uid','qid'])

In [10]:
from tqdm import tqdm 
tqdm.pandas()

In [11]:
ques_info=ques_info[['qid','qtime']]

In [12]:
answer_info=pd.merge(answer_info,ques_info,how='left',on='qid')

In [13]:
def time_(x):
    temp=x[1:].split('-H')
    return int(temp[0])*24+int(temp[1])

In [14]:
answer_info['aday']=answer_info['atime'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
answer_info['ahour']=answer_info['atime'].progress_apply(lambda x:int(x[1:].split('-H')[1]))
answer_info['atime']=answer_info['atime'].progress_apply(lambda x:time_(x))
answer_info['aweek']=answer_info['aday'].progress_apply(lambda x:x%7)

answer_info['qday']=answer_info['qtime'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
answer_info['qhour']=answer_info['qtime'].progress_apply(lambda x:int(x[1:].split('-H')[1]))
answer_info['qtime']=answer_info['qtime'].progress_apply(lambda x:time_(x))
answer_info['qweek']=answer_info['qday'].progress_apply(lambda x:x%7)


100%|██████████| 4694586/4694586 [00:07<00:00, 639143.30it/s]


In [15]:
answer_info.head()

,atime,qid,uid,qtime,aday,ahour,aweek,qday,qhour,qweek
0,91399,Q1867533817,M625498202,91365,3808,7,0,3806,21,5
1,91457,Q3366788616,M142330444,91401,3810,17,2,3808,9,0
2,92485,Q4264694221,M771499642,92060,3853,13,3,3835,20,6
3,92634,Q1088851650,M2282072267,78499,3859,18,2,3270,19,1
4,92542,Q1023877868,M2282072267,92227,3855,22,5,3842,19,6


In [16]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
all_data=pd.concat([train[['qid','uid','time']],test])

all_data['iday']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
all_data['ihour']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[1]))
all_data['itime']=all_data['time'].progress_apply(lambda x:time_(x))
all_data['iweek']=all_data['iday'].progress_apply(lambda x:x%7)

100%|██████████| 10630880/10630880 [00:16<00:00, 645205.73it/s]


In [17]:
all_data.drop(['time'],axis=1,inplace=True)

In [18]:
all_data.head()

,qid,uid,iday,ihour,itime,iweek
0,Q2166419046,M401693808,3865,22,92782,1
1,Q1550017551,M3392373099,3844,11,92267,1
2,Q604029601,M2317670257,3862,15,92703,5
3,Q2350061229,M1618461867,3849,11,92387,6
4,Q2443223942,M3544409350,3867,4,92812,3


In [19]:
all_data_uid=pd.merge(all_data,answer_info,on='uid',how='inner')
all_data_uid['sub_time']=all_data_uid['itime']-all_data_uid['atime']#邀请时间与之前回答的时间的差值
all_data_uid=all_data_uid[all_data_uid['sub_time']>0]

In [20]:
all_data_uid.head()

,qid_x,uid,iday,ihour,itime,iweek,atime,qid_y,qtime,aday,ahour,aweek,qday,qhour,qweek,sub_time
0,Q604029601,M2317670257,3862,15,92703,5,92364,Q3034061280,92364,3848,12,5,3848,12,5,339
2,Q604029601,M2317670257,3862,15,92703,5,92539,Q1309562090,92472,3855,19,5,3853,0,3,164
6,Q426016802,M2317670257,3860,13,92653,3,92364,Q3034061280,92364,3848,12,5,3848,12,5,289
8,Q426016802,M2317670257,3860,13,92653,3,92539,Q1309562090,92472,3855,19,5,3853,0,3,114
12,Q4274079416,M2317670257,3855,12,92532,5,92364,Q3034061280,92364,3848,12,5,3848,12,5,168


In [21]:
all_data_uid.drop(['qid_y'],axis=1,inplace=True)

In [22]:
all_data_uid['qaStime']=all_data_uid['atime']-all_data_uid['qtime']#过去回答时间与问题时1间差值

In [23]:
all_data_uid['qaShour']=all_data_uid['ahour']-all_data_uid['qhour']#过去回答时间与问题时间差值

In [24]:
all_data_uid['qaSday']=all_data_uid['aday']-all_data_uid['qday']#过去回答时间与问题时间差值

In [25]:
all_data_uid['sub_day']=all_data_uid['iday']-all_data_uid['aday']

In [26]:
feature=all_data_uid.groupby(['qid_x','uid']).agg({'sub_time':'min',#距离上次回答的小时
                                                    'sub_day':'min',#距离上次回答的天数
                                                    'ahour':['mean','median'],#过去喜欢几点回答
                                                    'aweek':['mean','median'],#过去喜欢哪周回答
                                                    'qaStime':['mean','max','min'],
                                                    'qaSday':['mean','max','min'],
                                                    'qaShour':['mean','max','min'],#提问与回答时间差值
                                                    'qtime':['mean','max','min'],#喜欢新问题or老问题
                                                    'qhour':['mean','max','min'],
                                                    'qweek':['mean','max','min','median'],
                                                   })

In [27]:
len(all_data)

10630880

In [28]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [29]:
import numpy as np

In [30]:
feature=reduce_mem_usage(feature)

Mem. usage decreased to 468.26 Mb (69.1% reduction)


In [31]:
feature=feature.reset_index()
feature.head()

qid_x          uid sub_time sub_day      ahour            aweek  \
                                 min     min       mean median      mean   
0  Q1000002524  M2358184554       41       2  17.000000   17.0  2.000000   
1  Q1000002524   M562088453      736      31  15.601562   16.0  2.400391   
2  Q1000006560  M1628216851       43       2  12.351562   11.0  2.980469   
3  Q1000007604  M2627631122      338      14  14.000000   14.0  6.000000   
4  Q1000007604   M274784249       16       1  17.671875   20.5  1.666992   

               qaStime         ...         qtime                    qhour      \
  median          mean    max  ...          mean    max    min       mean max   
0    2.0   4037.000000   4037  ...  88596.000000  88596  88596  12.000000  12   
1    2.0     65.400002    169  ...  91601.398438  91891  91451  17.406250  21   
2    4.0  16211.745117  57494  ...  75577.078125  92068  34101  13.312500  23   
3    6.0     47.000000     47  ...  91839.000000  91839  91839  15.000000  15   
4    1.5   4078.000000  18587  ...  87735.664062  91959  72922  15.664062  21   

          qweek                 
  min      mean max min median  
0  12  2.000000   2   2    2.0  
1  11  2.400391   6   0    2.0  
2   1  3.568359   6   0    4.0  
3  15  4.000000   4   4    4.0  
4  10  3.333984   6   0    3.0  

[5 rows x 27 columns]

In [32]:
feature.columns=['qid','uid','last_atime','last_aday','ahour_mean','ahour_median','aweek_mean','aweek_median',
                'qaStime_mean','qaStime_max','qaStime_min','qaSday_mean','qaSday_max','qaSday_min',
                'qaShour_mean','qaShour_max','qaShour_min','qtime_mean','qtime_max','qtime_min',
                'qhour_mean','qhour_max','qhour_min','qweek_mean','qweek_max','qweek_min','qweek_median']

In [34]:
feature.to_hdf('feature_atime_new.h5', key='data')